In [ ]:
#Installing libraries
!pip install transformers
!pip install  nlp
!pip install pyarrow==0.16.0

In [ ]:
import torch
import nlp
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

In [ ]:
config = T5Config.from_json_file('config.json')
tokenizer = T5Tokenizer.from_pretrained('model.ckpt.model')

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('small_model.bin',config=config)

In [ ]:
#Let's import the ground truth
import pandas as pd
import time
from random import sample
from tqdm import tqdm


SAMPLE_SIZE = 10

df = pd.read_csv('test_comment.tsv',header=None,sep='\t')


inputs = []

for item in df[0]:
    inputs.append('generate comment: %s' % item)



samp = sample(inputs,SAMPLE_SIZE)

items = [tokenizer.encode(item,return_tensors='pt') for item in samp]

In [ ]:
inputs[0]

In [ ]:
BEAM_SIZE = [1,5,10,25,50]

with open('INF_TIME_comment.txt','w') as fwrite:


    for beam in tqdm(BEAM_SIZE):

        acc=0

        for idx in range(0,SAMPLE_SIZE):

                start = time.time()

                beam_output = model.generate(
                  items[idx],
                  max_length=512, 
                  num_beams=beam, 
                  early_stopping=True
                )

                end = time.time()

                acc += (end-start)

        fwrite.write('Average inference time on beam width k=%s: %s\n' % (beam,str(acc/SAMPLE_SIZE)))
    
    fwrite.close()


In [ ]:
with open('inference_samples.txt','w') as writeSam:
    for item in samp:
        writeSam.write(item+'\n')
    writeSam.close()
    